# Some SPARQL queries
<div style="color: red; font-style: italic;">Note: when viewed through <span style="font-style: normal; font-weight: bold;">nbviewer</span> or converted through <span style="font-style: normal; font-weight: bold;">nbconvert</span> this notebook looks slightly different than when rendered by a running <em>sparqlkernel</em>. See a <a href="http://htmlpreview.github.io/?https://github.com/paulovn/sparql-kernel/blob/master/examples/sparql-endpoints.html">converted version</a> for an example more similar to the actual look in a "live" kernel.</div>


We are going to use some of the [Public SPARQL endpoints](https://www.w3.org/wiki/SparqlEndpoints). Let's look first at the set of available magics

In [1]:
%lsmagics

Available magics:
%auth  %display  %endpoint  %format  %graph  %header  %lang  %log  %lsmagics  %outfile  %prefix  %qparam  %show

%auth (basic|digest|none) <username> <passwd> : send HTTP authentication
%display raw | table [withtypes] | diagram [svg|png] [withliterals] : set display format
%endpoint <url> : set SPARQL endpoint. **REQUIRED**
%format JSON | N3 | XML | any | default : set requested result format
%graph <uri> : set default graph for the queries
%header <string> | OFF : add a persistent header line before the query, or delete all defined headers
%lang <lang> [...] | default | all : language(s) preferred for labels
%log critical | error | warning | info | debug : set logging level
%lsmagics  : list all magics
%outfile <filename> | NONE : save raw output to a file (use "%d" in name to add cell number, "NONE" to cancel saving)
%prefix <name> [<uri>] : set (or delete) a persistent URI prefix for all queries
%qparam <name> [<value>] : add (or delete) a persistent custom parameter to the endpoint query
%show <n> | all : maximum number of shown results

We set a few default options via those magics.

In [2]:
# Modify output format
# Don't show more than 80 results (event if more are fetched)
%show 80

# Request whatever format is appropriate for the query type
%format default

# Activate table output
%display table

Result maximum size: 80
Return format: DEFAULT
Display: table

## DBPedia

Note that DBPedia has a set of [predefined namespace prefixes](http://es.dbpedia.org/sparql?nsdecl) that we can use without the need to define them in the query.

### Simple queries

Find a few predicates that can point to people

In [3]:
%endpoint http://dbpedia.org/sparql
%display table withtypes

SELECT DISTINCT ?property
WHERE {
   ?s ?property ?person .
   ?person rdf:type foaf:Person .
}
LIMIT 10

Endpoint set to: http://dbpedia.org/sparql
Display: table
Show Types: on

property,type
http://www.w3.org/2002/07/owl#differentFrom,uri
http://www.w3.org/2000/01/rdf-schema#seeAlso,uri
http://www.w3.org/2002/07/owl#sameAs,uri
http://dbpedia.org/ontology/wikiPageWikiLink,uri
http://dbpedia.org/ontology/wikiPageRedirects,uri
http://dbpedia.org/ontology/wikiPageDisambiguates,uri
http://dbpedia.org/property/musicalguests,uri
http://dbpedia.org/property/avpm,uri
http://dbpedia.org/property/name,uri
http://xmlns.com/foaf/0.1/primaryTopic,uri


Let's use the `dbp:successor` predicate to find chains of at least 4 persons that form a sequence. Since it can produce a nice graph, let's output it as a diagram.

*(note: formerly using the `dbp:before` property, but that one seems not to be available anymore for DBPedia entries)*

In [5]:
%endpoint http://dbpedia.org/sparql
%display diagram

CONSTRUCT {
    ?p1 dbp:successor ?p2 .
    ?p2 dbp:successor ?p3 .
    ?p3 dbp:successor ?p4 .
}
WHERE {
    ?p1 rdf:type foaf:Person .
    ?p1 dbp:successor ?p2 .
    ?p2 dbp:successor ?p3 .
    ?p3 dbp:successor ?p4 .
}
LIMIT 50

Endpoint set to: http://dbpedia.org/sparql
Display: svg

Error: Response processing error: Exception while drawing graph: FileNotFoundError(2, "No such file or directory: 'dot'")

A `DESCRIBE` query, also represented as a diagram.

In [ ]:
%endpoint http://dbpedia.org/sparql
%display diagram
%lang es
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
DESCRIBE <http://dbpedia.org/resource/Asturias>
LIMIT 10

### Elaborate queries

#### Example 1
This one is taken from [SPARQL By Example](https://www.w3.org/2009/Talks/0615-qbe/#q5):

* Find out all countries without coastline and population greater than 15M. 
* Ordered by population

_Modified: former `type:LandlockedCountries` property moved to `yago:WikicatLandlockedCountries`_

In [ ]:
%endpoint http://dbpedia.org/sparql
%format default
%display table 
%lang all

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>        
PREFIX yago: <http://dbpedia.org/class/yago/>
PREFIX prop: <http://dbpedia.org/property/>
SELECT ?country_name ?population
WHERE {
    ?country a yago:WikicatLandlockedCountries ;
             rdfs:label ?country_name ;
             prop:populationEstimate ?population .
    FILTER (?population > 15000000) .
    FILTER (langMatches(lang(?country_name), "EN")) .
} ORDER BY DESC(?population)

#### Example 2
Where was Henrik Ibsen born? (taken from a [StackOverflow answer](http://stackoverflow.com/a/21859115)).

In [ ]:
%endpoint http://dbpedia.org/sparql
%display table withtypes
%lang all

PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>
SELECT ?label { 
 dbr:Henrik_Ibsen
   dbpedia-owl:birthPlace
     [ a dbpedia-owl:Country ;
       rdfs:label ?label ]
 FILTER langMatches(lang(?label),"en")
}

## Wikidata
Let's use now the [wikidata](http://wikidata.org) SPARQL endpoint query service. You can read its [user manual](https://www.mediawiki.org/wiki/Wikidata_query_service/User_Manual). There is also a page with many [example queries](https://www.mediawiki.org/wiki/Wikibase/Indexing/SPARQL_Query_Examples)

### Example 1
This was taken from [Causes of death for US presidents](http://ramiro.org/notebook/us-presidents-causes-of-death/)

In [ ]:
%endpoint http://query.wikidata.org/sparql
%display table
%show all

PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?president ?cause ?dob ?dod WHERE {
    ?pid wdt:P39 wd:Q11696 .
    ?pid wdt:P509 ?cid .
    ?pid wdt:P569 ?dob .
    ?pid wdt:P570 ?dod .

    OPTIONAL {
        ?pid rdfs:label ?president filter (lang(?president) = "en") .
    }
    OPTIONAL {
        ?cid rdfs:label ?cause filter (lang(?cause) = "en") .
    }
}

### Example 2
Birthdays for today. We fetch only the oldest 20 (depending on the date, there could be many). Example from the [wikidata example list](https://www.mediawiki.org/wiki/Wikibase/Indexing/SPARQL_Query_Examples#Whose_birthday_is_today.3F)

In [ ]:
%endpoint http://query.wikidata.org/sparql

SELECT ?entityS ?entity (year(?date) as ?year) 
WHERE 
{
    ?entityS wdt:P569 ?date .
    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
        ?entityS rdfs:label ?entity
    } 
    FILTER (datatype(?date) = xsd:dateTime && month(?date) = month(now()) && day(?date) = day(now()))
}
ORDER BY ASC(?year)
LIMIT 20

### Example 3

Who discovered the most asteroids?. Taken also from the [example page](https://www.mediawiki.org/wiki/Wikibase/Indexing/SPARQL_Query_Examples#Who_discovered_the_most_asteroids.3F)

In [ ]:
%endpoint http://query.wikidata.org/sparql
%display table
SELECT ?discoverer ?name (COUNT(?asteroid) AS ?count)
WHERE
{
    ?asteroid wdt:P31 wd:Q3863 .
    ?asteroid wdt:P61 ?discoverer .
    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
        ?discoverer rdfs:label ?name
    }
}
GROUP BY ?discoverer ?name
ORDER BY DESC(?count)
LIMIT 10

## Linkedmdb

[linkedMDB](http://data.linkedmdb.org/) contains an RDF database for Movies. It also has a web-based [query interface](http://data.linkedmdb.org/snorql/)

Let's find movie titles with the `ring` word in it.

In [ ]:
%endpoint http://data.linkedmdb.org/sparql
%format json
%display table

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX movie: <http://data.linkedmdb.org/resource/movie/>
SELECT ?movie ?label
WHERE {
    ?movie rdf:type movie:film .
    ?movie rdfs:label ?label .
    FILTER regex(?label, "ring", "i")
} LIMIT 10


<div style="border-top: 1px gray solid; font-family: Cursive, sans-serif;">Version: 1.2 (2017-03-31)<br/>
Author: Paulo Villegas</div>